# Présentation de la boîte à outil Julia pour l'algèbre Max-Plus

L'algebre Max-Plus (max+) redéfinit les opérateurs addition et multiplication de l'algèbre classique par respectivement les opérateurs maximum et addition :

$$a \oplus b \triangleq \max(a,b)$$
$$a \otimes b \triangleq a + b$$

L'algebre Max-Plus permet de modéliser et d'évaluer les performances de systèmes à évènements discrets (réseaux de transport ou de télécom, systèmes de production), mais également dans des domaines tels que la théorie de la décision, recherche opérationnelle ...

Une boîte à outil Max-Plus pour le langage Julia version >= 1.0.3 peut être téléchargée depuis ce repo GitHub https://github.com/Lecrapouille/MaxPlus.jl ou bien depuis le système de paquets de Julia via la commande `] add MaxPlus`. Cette boîte à outil est un portage non officiel de celle qui état intégrée dans le logiciel Scilab, puis par la suite, dans le logiciel fork [ScicosLab](http://www.scicoslab.org) maintenu à l'époque par les anciens chercheurs de Scilab mais qui est désormais remplacé par le second fork [NSP](https://cermics.enpc.fr/~jpc/nsp-tiddly/mine.html).

Ce document permet à la fois d'introduire l'algèbre Max-Plus tout en présentant les fonctions de base de cette boîte à outils Julia. Pour ceux qui maîtrisent déjà cette algèbre, peuvent aller directement consulter cette [bibliographie](../docs/src/bibliography.md). Pour ceux qui désirent comparer les résultats obtenus avec ceux fournis par Sicoslab on rappellera qu'un nombre Max-Plus Sicoslab est créé avec la notation suivante `#(42)`, que les éléments neutres et absorbants obtenus par `%0` et `%1` et qu'une démonstration interactive de la boîte à outil peut être lancée depuis leur menu.

## Installation de la boîte à outil Julia

Tout d'abord vérifions la version de votre Julia. Cette boîte à outils Max-Plus devrait fonctionner avec version obsolète Julia 0.4 mais privilégiez au moins une version supérieure à 1.3 car elle apporte des correctifs sur les matrices creuses et si certains de mes propositions de correctifs pour Julia n'ont toujours pas été pris en compte ils sont appliqués dans ce paquet Max-Plus.

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: AMD Ryzen 7 1800X Eight-Core Processor
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, znver1)


Ensuite, nous allons installer la boîte à outils Max-Plus Julia. Vous avez plusieurs options pour le faire. Les codes suivants ne fonctionnent pas directement depuis ce document Jupyter, pensez donc à les décommenter et de les exécuter depuis le mode interactif Julia (REPL) :
- vous pouvez télécharger la version du code source depuis GitHub :

In [2]:
# using Pkg; Pkg.add(PackageSpec(url="https://github.com/Lecrapouille/MaxPlus.jl"));
# using MaxPlus;

- Soit depuis le gestionnaire de paquet Julia (version stable):

In [3]:
# using Pkg; Pkg.add("MaxPlus")

## Utiliser Max-Plus depuis un document Jupyter

Une fois le paquet Julia Max-Plus installé depuis le REPL Julia, lancez un notebook Jupyter et ouvrez cette page ci via les commandes REPL suivantes :

In [4]:
# using IJulia
# notebook()

Depuis ce document Jupyter (celui que vous lisez), chargez la boîte à outil Max-Plus depuis le dossier MaxPlus.jl :

In [5]:
push!(LOAD_PATH, pwd())
using MaxPlus

┌ Info: Precompiling MaxPlus [41177cfe-c387-11e9-2806-edd030e4594e]
└ @ Base loading.jl:1317


Pour le moment et dans un soucis pédagogique on active un mode d'affichage particulier des nombres Max-Plus. L'explication sera donnée plus tard.

In [6]:
# Sur Jupyter le mode LaTeX est forcé ce qu'on ne veut pas !
# On force l'affichage en texte plein.
Base.show(io::IO, ::MIME"text/latex", x::MP{T}) where T = show(io, MIME"text/plain", x)
Base.show(io::IO, ::MIME"text/latex", A::ArrMP{T}) where T = show(io, MIME"text/plain", A)
Base.show(io::IO, ::MIME"text/latex", S::SpaMP{T}) where T = show(io, MIME"text/plain", S)

# Force l'affichage des nombres Max-Plus pour afficher les -infini
mp_change_display(0);

## Les types Max-Plus pour Julia

Avant de présenter l'algèbre Max-Plus, créons quelques nombres Max-Plus sous Julia. Par exemple on écrira :

In [7]:
a = MP(1.0)
b = MP(3.0)
c = MP(5)
typeof(a), typeof(b), typeof(c)

(MP{Float64}, MP{Float64}, MP{Int64})

Mes nombres max+ sont en fait templatés (Float ou Int) mais Julia déduit directement leur type. On peut éventuellement forcer leur type : 

In [8]:
a = MP{Float64}(42)
a, typeof(a)

(42.0, MP{Float64})

Même idée pour les entiers (bien que les entiers soient moins recommandés) :

In [9]:
b = MP{Int64}(3)
b, typeof(b)

(3, MP{Int64})

Les nombres max+ contaminent sur les autres nombres (entiers, réels) : ils convertissent un nombre non max+ en nombre max+ via les opérateurs arithmétiques où opérateurs de promotion implicites :

In [10]:
d = 1.0
typeof(c), typeof(d), typeof(c + d)

(MP{Int64}, Float64, MP{Float64})

Nous voyons que l'addition Max-Plus à transformé la variable `c` MP{Int64} en MP{Float64}. Même idée pour les nombres entiers :

In [11]:
f = 1
typeof(c), typeof(f), typeof(c + f)

(MP{Int64}, Int64, MP{Int64})

La contamination fonctionne également sur les matrices denses et creuses :

In [12]:
[MP(1.0) 2; 3 4]

2×2 Matrix{MP{Float64}}:
  1.0   2.0
  3.0   4.0


MP(1.0) de type MP{Float64} a contaminé les nombres entiers *classiques* 2 3 et 4 en MP{Float64}. Voici une façon plus simple de le faire :

In [13]:
MP([1.0 2; 3 4])

2×2 Matrix{MP{Float64}}:
  1.0   2.0
  3.0   4.0


Voici un autre exemple plus complexe de matrice contenant des nombres infinis $-\infty$ (qui sont du type MP{Float}):

In [14]:
I = MP([-Inf 0; 0 -Inf])

2×2 Matrix{MP{Float64}}:
  -Inf    0.0
   0.0   -Inf


Si l'on désire convertir un nombre Max-Plus en un nombre ordinaire (au sens algèbre classique, celle que l'on utilise tous les jours) :

In [15]:
b = plustimes(a)
typeof(a), typeof(f), typeof(MP(b)), typeof(plustimes(MP(b)))

(MP{Float64}, Int64, MP{Float64}, Float64)

Nous voyons que la variable `a` étant du type MP{Float64}, `b` est devenu de type Float64. Elle se comportera désormais comme un nombre de l'algèbre classique.

In [16]:
[f       a
 f + f   a + a]

2×2 Matrix{MP{Float64}}:
  1.0   42.0
  2.0   42.0


Vu que `a` est de MP{Float64} il contamine `e` et la matrice est du type MP{Float64}.

On peut vouloir convertir les nombres Max-Plus en nombre de l'algèbre Min-Plus (où les signes des $-\infty$ sont inversés) bien que le type soit toujours du type Max-Plus :

In [17]:
p = minplus(MP(-Inf))
p, typeof(p)

(Inf, MP{Float64})

## Les constantes Max-Plus pour Julia

Certaines constantes (éléments neutres et absorbants que l'on verra plus tard) Max-Plus sont prédéfinies :
- mp0 pour $-\infty$ (que l'on note en général $\varepsilon$ ou parfois $\mathbb{0}$),
- mp1 pour 0 (que l'on note en général $e$ ou parfois $\mathbb{1}$),
- mptop pour $+\infty$ (utilisé pour l'algèbre Min-Plus).

In [18]:
mp0, ϵ, mp1, mptop, mpzero(), mpone(), mptop

(-Inf, -Inf, 0.0, Inf, -Inf, 0.0, Inf)

Le template peuvent être ajouté :

In [19]:
mpzero(Int64), mpone(Int64), mpzero(Float64), mpone(Float64)

(-9223372036854775808, 0, -Inf, 0.0)

Mais attention mp0, mp1, $\epsilon$, `e` et mptop sont prédéfinis avec le type MP{Float}.

## Affichage des nombres Max-Plus

Il y a 4 styles possibles d'affichage des nombres Max-Plus que l'on peut contrôler avec la fonction `mp_change_display`, le mode $1$ étant celui défini par défaut :
- Style 0: les nombres $-\infty$ et les $0$ sont affichés tel quel.
- Style 1 ou 2: les nombres $-\infty$ sont affiché sous la forme d'un point.
- Style 3 ou 4: les nombres $-\infty$ sont affichés sous la forme $\varepsilon$.
- Style 1 ou 3: les zéros sont affichés $0$.
- Style 2 ou 4: les zéros sont affichés $e$.

Notons que les nombres réels qui peuvent être écrits comme des entiers seront affichés comme des entiers et que le style par défaut est le $1$ car il correspond à l'affichage de ScicosLab car ils permettent d'afficher les matrices creuses de façon compacte. En effet, il est commun en Max-Plus de devoir manipuler et afficher de grosses matrices creuses.

In [20]:
mp_change_display(0)
I = MP([-Inf 0; 0 -Inf])

2×2 Matrix{MP{Float64}}:
  -Inf    0.0
   0.0   -Inf


In [21]:
mp_change_display(2)
I

2×2 Matrix{MP{Float64}}:
  .   e
  e   .


In [22]:
mp_change_display(3)
I

2×2 Matrix{MP{Float64}}:
  ε   0
  0   ε


In [23]:
mp_change_display(4)
I

2×2 Matrix{MP{Float64}}:
  ε   e
  e   ε


In [24]:
# Le mode par défaut :
mp_change_display(1)
I

2×2 Matrix{MP{Float64}}:
  .   0
  0   .


A partir d'une matrice Max-Plus, on peut générer le code $\LaTeX$ grâce à la fonction `LaTeX` ou via la fonction `show` avec l'argument `MIME"text/latex"`. La fonction `mp_change_display` modifie en conséquence le code LaTeX généré.

In [25]:
mp_change_display(0)
LaTeX(stdout, I)

\left[
\begin{array}{*{20}c}
-\infty & 0 \\
0 & -\infty \\
\end{array}
\right]


Une fois ce code $\LaTeX$ compilé, il affichera :

$$\left[
\begin{array}{*{20}c}
-\infty & 0 \\
0 & -\infty \\
\end{array}
\right]$$

Alors que :

In [26]:
mp_change_display(2)
LaTeX(stdout, I)

\left[
\begin{array}{*{20}c}
. & e \\
e & . \\
\end{array}
\right]


Une fois ce code $\LaTeX$ compilé, il affichera :

$$\left[
\begin{array}{*{20}c}
\varepsilon & e \\
e & \varepsilon \\
\end{array}
\right]$$

## Opérateur Max-Plus $\oplus$

L'opérateur addition redéfini par l'opérateur max() de l'algèbre classique. Son symbole, pour le différencier de l'addition dans l'algèbre classique, est $\oplus$. Mais en Julia on gardera le symbole `+`. Cet opérateur est associatif, commutatif, a un élément neutre (noté $\varepsilon$) et est idempotent.

$$a \oplus b \triangleq \max(a,b)$$

In [27]:
mp_change_display(1);
a = MP(1.0); b = MP(3.0); c = MP(5.0);
(a, b, c)

(1, 3, 5)

In [28]:
a + b    # ≜ max(a, b) == max(1, 3) == 3

MP{Float64}:
  3

#### Commutativité de $\oplus$

$$a \oplus b = b \oplus a$$
$$\triangleq$$
$$\max(a,b) = \max(b,a)$$

In [29]:
a + b == b + a

true

#### Associativité de $\oplus$

$$a \oplus b \oplus c = (a \oplus b) \oplus c = a \oplus (b \oplus c)$$

In [30]:
a + b + c == (a + b) + c == a + (b + c)

true

In [31]:
a + b + c # ≜ max(a, b, c) == max(1, 3, 5)

MP{Float64}:
  5

#### Elément neutre $\varepsilon$ pour $\oplus$

$$a \oplus \varepsilon = \varepsilon \oplus a = a$$
$$\triangleq$$
$$\max(a,-\infty) = \max(-\infty,a) = a$$

In [32]:
a + mp0 == mp0 + a == a

true

In [33]:
a + ϵ == ϵ + a == a

true

In [34]:
(a, mp0, ϵ), (a + mp0, a + ϵ), (mp0 + a, ϵ + a)

((1, ., .), (1, 1), (1, 1))

Notons que 0 est neutre pour les nombres positifs :

In [35]:
a + 0 == 0 + a == a

true

In [36]:
a, 0, a + 0

(1, 0, 1)

#### Elément absorbant $\infty$ pour $\oplus$

$$a \oplus \infty = \infty \oplus a$$
$$\triangleq$$
$$\max(a,\infty) = \max(\infty,a) = \infty$$

In [37]:
a + mptop == mptop + a == mptop

true

In [38]:
a, mptop, a + mptop

(1, Inf, Inf)

#### $\oplus$ est idempotent

In [39]:
a + a    # ≜ max(a, a) == max(1, 1) == 1

MP{Float64}:
  1

## Opérateur Max-Plus $\otimes$

L'opérateur multiplication est redéfini par l'opérateur addition qui est associatif, commutatif, a l'élément neutre $e$, l'élément absorbant $\varepsilon$ et est distributif sur $\oplus$.

In [40]:
a * b    # ≜ a + b == 1 + 3 == 4

MP{Float64}:
  4

#### Commutativité de $\otimes$

$$a \otimes b = b \otimes a$$
$$\triangleq$$
$$a + b = b + a$$

In [41]:
a * b == b * a

true

#### Associativité de $\otimes$

$$a \otimes b \otimes c = (a \otimes b) \otimes c = a \otimes (b \otimes c)$$

In [42]:
a * b * c == (a * b) * c == a * (b * c)

true

In [43]:
a * b * c

MP{Float64}:
  9

#### Elément neutre $e$ pour $\otimes$

$$a \otimes e = e \otimes a = a$$
$$\triangleq$$
$$a + 0 = 0 + a = a$$

In [44]:
a * mp1 == mp1 * a == a

true

In [45]:
a * e == e * a == a

true

#### Elément absorbant $\varepsilon$ pour $\otimes$

$$a \otimes \varepsilon = \varepsilon \otimes a = \varepsilon$$
$$\triangleq$$
$$a -\infty = -\infty + a = -\infty$$

In [46]:
a * mp0 == mp0 * a == mp0

true

In [47]:
a * ϵ == ϵ * a == ϵ

true

Par convention:

$$\infty \otimes \varepsilon = \varepsilon \otimes \infty = \varepsilon$$

In [48]:
# mptop * mp0 # FIXME shall return mp0

**FIXME** Help wanted

#### $\otimes$ n'est pas idempotent

In [49]:
a * a    # ≜ a + a == 1 + 1 == 2

MP{Float64}:
  2

---
### Distributivité de $\otimes$ sur $\oplus$

$$a \otimes (b \oplus c) = (a \otimes b) \oplus (a \otimes c)$$
$$(b \oplus c) \otimes a = (b \otimes a) \oplus (c \otimes a)$$

$$a \oplus b \otimes c = a + \max(a, b)$$
$$a \otimes c \oplus b \otimes c = \max(a+c,b+c)$$

In [50]:
(a + b) * c == (a * c) + (b * c)     # => max(a, b) + c == max(a + c, b + c) 

true

In [51]:
(a * c) + (b * c)

MP{Float64}:
  8

## Produit matriciel

Les matrices peuvent être de type Max-Plus. Le produit matriciel correspond au produit matriciel avec les opérateurs $+$ et $\times$ surchargés.

$$A=\begin{bmatrix}
4 & 3 \\
7 & -\infty
\end{bmatrix}\;,$$

$$A \otimes A = \begin{bmatrix}
4 \otimes 4 \oplus 3 \otimes7 & 4 \otimes 3 \oplus 3 \otimes -\infty \\
7 \otimes 4 \oplus -\infty \otimes 7 & 7 \otimes 3 \oplus -\infty \otimes -\infty
\end{bmatrix}\; = \begin{bmatrix}
10 & 7 \\
11 & 10
\end{bmatrix}\; = A^2.$$

In [52]:
A = MP([4 3; 7 -Inf])
A * A

2×2 Matrix{MP{Float64}}:
  10    7
  11   10


In [53]:
A * A == A^2

true

En algèbre Max-Plus l'opérateur puissance se comporte comme une multiplication dans l'algèbre classique :

In [54]:
MP(2.0)^5   # ==> 2 * 5

MP{Float64}:
  10

In [55]:
MP(2.0)^0   # ==> 2 * 0

MP{Float64}:
  0

S'applique également aux matrices :

In [56]:
A^5

2×2 Matrix{MP{Float64}}:
  24   23
  27   24


In [57]:
A^0

2×2 Matrix{MP{Float64}}:
  0   .
  .   0


## Quelques matrice utiles

### Matrice dense d'identité

Taille 2 $\times$ 2 :

$$\left[
\begin{array}{*{20}c}
e & \varepsilon \\
\varepsilon & e \\
\end{array}
\right]$$

In [58]:
I = mpeye(2,2) # Equivalent à : mpeye(Float64, 2,2)

2×2 Matrix{MP{Float64}}:
  0   .
  .   0


In [59]:
I = mpeye(3) # Equivalent à : mpeye(3,3)

3×3 Matrix{MP{Float64}}:
  0   .   .
  .   0   .
  .   .   0


Taille 3 $\times$ 2 :

In [60]:
I = mpeye(3,2)

3×2 Matrix{MP{Float64}}:
  0   .
  .   0
  .   .


### Matrices denses remplies uniquement de $e$ :

$$\left[
\begin{array}{*{20}c}
e & e \\
e & e \\
\end{array}
\right]$$

In [61]:
O = mpones(2,2) # Equivalent à : mpones(Float64, 2,2)

2×2 Matrix{MP{Float64}}:
  0   0
  0   0


In [62]:
O = mpones(2,3)

2×3 Matrix{MP{Float64}}:
  0   0   0
  0   0   0


In [63]:
O = mpones(2)

2×1 Matrix{MP{Float64}}:
  0
  0


### Matrices creuses remplies de $\varepsilon$ :

$$\left[
\begin{array}{*{20}c}
\varepsilon & \varepsilon \\
\varepsilon & \varepsilon \\
\end{array}
\right]$$

In [64]:
Z = mpzeros(2,2) # Equivalent à mpzeros(Float64, 2,2)

2×2 SparseMatrixCSC{MP{Float64}, Int64} with 0 stored entries:
  .   .
  .   .


In [65]:
Z = mpzeros(2,3)

2×3 SparseMatrixCSC{MP{Float64}, Int64} with 0 stored entries:
  .   .   .
  .   .   .


In [66]:
Z = mpzeros(2)

2-element SparseArrays.SparseVector{MP{Float64}, Int64} with 0 stored entries

### Matrices denses remplies de $\varepsilon$ :

In [67]:
Z = full(mpzeros(Float64, 2,2))

2×2 Matrix{MP{Float64}}:
  .   .
  .   .


**Note :** On notera la présence de la fonction full() (ou son alias dense()) pour convertir une matrice creuse en matrice pleine (dense).

## Matrices creuses

Une matrice creuse est une matrice contenant beaucoup de zéros. En algèbre classique les zéros sont 0 (pour les entiers) ou 0.0 (réels) mais en Max-Plus les zéros valent $-\infty$. La structure interne des matrices creuses est conçu pour ne pas garder en mémoire ces zéros (sauf si demandé explicitement). Pour créer une matrice creuse en Julia il faut donner trois vecteurs : un vecteur pour garder les données non nulles et deux vecteurs pour mémoriser les index de ces données.

Beaucoup d'exemples dans la nature peuvent être représenté par des matrices creuses plutôt que par des matrices pleines comme par exemple le réseau routier est un graphe où les routes sont les arcs et les carrefours sont les noeuds. Les graphes sont représentés en mémoire soit sous forme de liste d'ajacence soit sous forme de matrice. Comme en général un carrefour permet de rejoindre entre 2 à 4 routes et que par conséquent jamais il n'existera une ville où tous les carrefours seront lés les uns aux autres il est donc préférable d'utiliser une matrice creuse. Mais à cause des leur index, les algorithmes sur les matrices seront plus pénalisés en temps d'exécution que pour les matrices pleines.

Pour la suite de ce document on aura besoin d'un paquet Julia de base supplémentaire :

In [68]:
using SparseArrays

**Note: On préférera la version Julia > 1.3 qui corrige des bogues dans les matrices creuses. Les matrices creuses Max-Plus ne stockent pas les valeurs $-\infty$ mais dans les versions précédentes de Julia les valeurs zéros étaient écrites sous la forme littérale $0$ au lieu de la fonction zero() entraînant des cas d'erreurs. Le paquet MaxPlus en corrige certains mais il n'est pas garanti qu'il les corrige tous ou n'entraine pas des nouveaux !**

Création d'une matrice creuse de taille 2x2 Max-Plus vide :

In [69]:
Z = mpzeros(2,2) # ou son équivalent mpzeros(Float64, 2,2)

2×2 SparseMatrixCSC{MP{Float64}, Int64} with 0 stored entries:
  .   .
  .   .


Création d'une matrice creuse Max-Plus via des données non Max-Plus:

In [70]:
MP(sparse([1, 2, 3], [1, 2, 3], [5, 2, 6]))

3×3 SparseMatrixCSC{MP{Int64}, Int64} with 3 stored entries:
  5   .   .
  .   2   .
  .   .   6


Création d'une matrice creuse Max-Plus avec des données Max-Plus :

In [71]:
sparse([1, 2, 3], [1, 2, 3], MP([5, 2, 6]))

3×3 SparseMatrixCSC{MP{Int64}, Int64} with 3 stored entries:
  5   .   .
  .   2   .
  .   .   6


Julia autorise de créer des matrices stockant les éléments zéros qui ils sont explicitement donnés :

In [72]:
A = MP(sparse([1, 2, 3], [1, 2, 3], [-Inf, 2, 0]))

3×3 SparseMatrixCSC{MP{Float64}, Int64} with 3 stored entries:
  .   .   .
  .   2   .
  .   .   0


Ici le -Inf n'est pas affiché mais il est bien stocké (3 stored entries).

In [73]:
mp_change_display(0);
A.nzval      # nzval = Non-Zero values

3×1 Matrix{MP{Float64}}:
  -Inf
   2.0
   0.0


Mais il n'est pas utilisé. Par exemple :

In [74]:
B = MP(sparse([2, 3], [2, 3], [2.0, 0]))

3×3 SparseMatrixCSC{MP{Float64}, Int64} with 2 stored entries:
  .   .   .
  .   2   .
  .   .   0


In [75]:
B.nzval

2×1 Matrix{MP{Float64}}:
  2.0
  0.0


In [76]:
A == B

true

Le $-\infty$ n'est pas testé. Si l'on ne désire pas stocker les $\epsilon$ on peut ajouter le paramètre `preserve=false` :

In [77]:
C = MP(sparse([1, 2, 3], [1, 2, 3], [-Inf, 2, 0]), keepzeros=false)

3×3 SparseMatrixCSC{MP{Float64}, Int64} with 2 stored entries:
  .   .   .
  .   2   .
  .   .   0


In [78]:
C.nzval

2×1 Matrix{MP{Float64}}:
  2.0
  0.0


In [79]:
A == B == C

true

### Conversion d'une matrice creuse en matrice pleine :

Les trois fonctions produisent le même résultat :

In [80]:
full(Z), dense(Z), Array(Z)

(MP{Float64}[-Inf -Inf; -Inf -Inf], MP{Float64}[-Inf -Inf; -Inf -Inf], MP{Float64}[-Inf -Inf; -Inf -Inf])

In [81]:
full(Z)

2×2 Matrix{MP{Float64}}:
  -Inf   -Inf
  -Inf   -Inf


On remarquera que cette matrice contient que des $-\infty$. En effet, ils correspondent aux 0 éliminés des matrices creuses en algèbre classique. Une matrice creuse Max-Plus ne stocke pas les nombres Max-Plus $-\infty$ (**note:** enfin jusqu'à Julia > 1.3 car les versions précédentes avaient un bogue elles confondaient 0 et zero(T) avec T template de type MP). 

Conversion d'une matrice creuse Max-Plus à partir d'une matrice pleine en algèbre classique :

In [82]:
A = [4.0 0; 7 5]

2×2 Matrix{Float64}:
 4.0  0.0
 7.0  5.0

In [83]:
B = mpsparse(A)

2×2 SparseMatrixCSC{MP{Float64}, Int64} with 3 stored entries:
  4   .
  7   5


On remarquera que le $-\infty$ a disparu. Si on voulait le garder : **FIXME**

In [84]:
mpsparse([4 0; 7 -Inf])

2×2 SparseMatrixCSC{MP{Float64}, Int64} with 3 stored entries:
  4   .
  7   .


In [85]:
C = sparse(MP(A))

2×2 SparseMatrixCSC{MP{Float64}, Int64} with 4 stored entries:
  4   0
  7   5


In [86]:
B.nzval, C.nzval

(MP{Float64}[4.0, 7.0, 5.0], MP{Float64}[4.0, 7.0, 0.0, 5.0])

## Conversion de matrices Max-Plus

### Max-Plus vers Min-Plus

On peut vouloir convertir les valeurs pour l'algèbre Min-Plus (les signes des $-\infty$ sont inversés) pour les scalaires, matrices creuses et pleines :

In [87]:
# Scalaires :
mp0, minplus(mp0)

(-Inf, Inf)

In [88]:
# Matrices creuses :
Z, minplus(Z)

(
  ⋅     ⋅  
  ⋅     ⋅  , 
 Inf  Inf
 Inf  Inf)

In [89]:
# Matrices pleines :
A = MP([4 0; 7 -Inf])
MP(A), minplus(MP(A))

(MP{Float64}[4.0 0.0; 7.0 -Inf], MP{Float64}[4.0 0.0; 7.0 Inf])

### Max-Plus vers algèbre classique

On peut vouloir convertir une matrice Max-Plus en valeurs non Max-Plus :

In [90]:
# Scalaires :
typeof(mp0), typeof(plustimes(mp0))

(MP{Float64}, Float64)

In [91]:
plustimes(Z)

2×2 SparseMatrixCSC{Float64, Int64} with 0 stored entries:
  ⋅    ⋅ 
  ⋅    ⋅ 

In [92]:
plustimes(MP(A))

2×2 Matrix{Float64}:
 4.0    0.0
 7.0  -Inf

## Calcul sur les matrices
### Trace d'une matrice

La trace est la somme Max-Plus des éléments diagonaux.

In [93]:
mptrace(A)

MP{Float64}:
  4.0

In [94]:
mptrace(A) == A[1,1] + A[2,2]

true

### Norme d'une matrice

In [95]:
mpnorm(mpeye(Float64, 2,2))

Inf

### Valeurs propres d'une matrice

In [96]:
A = mpsparse([1.0 2; 3 4])
l,v = howard(A)

PI=[2, 2]C=[2.0, 4.0]vaux=[2.0, 4.0]Visiting from node 1 color=1
*** ITERATION 0 of Max Plus Howard Algorithm *** 
PI=[2, 2]
c=[2.0, 4.0]
chi=[4.0, 4.0]
v=[2.0, 4.0]


(MP{Float64}[4.0, 4.0], MP{Float64}[2.0, 4.0])

In [103]:
A

2×2 SparseMatrixCSC{MP{Float64}, Int64} with 4 stored entries:
  1   2
  3   4


In [108]:
full(A)*v == l[1]*v

true

In [110]:
A*v, full(A)*v # FIXME BUG JULIA

(MP{Float64}[7.0, 9.0], MP{Float64}[6.0, 8.0])

In [116]:
full(A)*A

2×2 Matrix{MP{Float64}}:
  6.0   7.0
  8.0   9.0


In [114]:
A*full(A)

2×2 Matrix{MP{Float64}}:
  6.0   7.0
  8.0   9.0


In [115]:
A*A

2×2 SparseMatrixCSC{MP{Float64}, Int64} with 4 stored entries:
  5   6
  7   8


---
### Algorithme $A^*$ (A star)